In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, GRU, Input, Concatenate, Reshape


data = pd.read_csv('data.csv')


label_enc = LabelEncoder()
data['Player Role'] = label_enc.fit_transform(data['Player Role'])
data['Team'] = label_enc.fit_transform(data['Team'])

features = ['Batting Average', 'Bowling Average', 'Strike Rate', 'Economy Rate', 
            'Centuries Scored', 'Half Centuries Scored', 'Ducks Scored', 
            'Wickets Taken Last Match', 'Runs Scored Last Match', 
            'Player Age', 'Player Experience', 'Player Role', 'Team']

X = data[features]
y = data[['Runs Scored', 'Wickets Taken', 'Balls Faced', 'Balls Bowled', 
          'Overs Bowled', 'Maidens Bowled', 'Runs Conceded']]


for col in y.columns:
    y[col] = y[col] / data['Player Experience']

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


C:\Users\shoya\AppData\Local\Temp\ipykernel_2640\1919140137.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[col] = y[col] / data['Player Experience']


In [2]:
def create_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    
    reshaped_input = Reshape((input_shape, 1))(input_layer)
    
    lstm = LSTM(64, return_sequences=True)(reshaped_input)
    lstm = LSTM(32)(lstm)
    
    gru = GRU(64, return_sequences=True)(reshaped_input)
    gru = GRU(32)(gru)
    
    concat = Concatenate()([lstm, gru])
    
    dense1 = Dense(64, activation='relu')(concat)
    dense2 = Dense(32, activation='relu')(dense1)
    
    output = Dense(7)(dense2)  
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model


model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=50, batch_size=4, validation_data=(X_test, y_test))



Epoch 1/50


6/6 [==============================] - 15s 560ms/step - loss: 0.9945 - mae: 0.7282 - val_loss: 1.0174 - val_mae: 0.7572
Epoch 2/50
6/6 [==============================] - 0s 28ms/step - loss: 0.9752 - mae: 0.7154 - val_loss: 1.0212 - val_mae: 0.7607
Epoch 3/50
6/6 [==============================] - 0s 31ms/step - loss: 0.9609 - mae: 0.7050 - val_loss: 1.0223 - val_mae: 0.7689
Epoch 4/50
6/6 [==============================] - 0s 25ms/step - loss: 0.9394 - mae: 0.6919 - val_loss: 1.0212 - val_mae: 0.7693
Epoch 5/50
6/6 [==============================] - 0s 23ms/step - loss: 0.9172 - mae: 0.6756 - val_loss: 1.0160 - val_mae: 0.7652
Epoch 6/50
6/6 [==============================] - 0s 23ms/step - loss: 0.8958 - mae: 0.6567 - val_loss: 1.0114 - val_mae: 0.7633
Epoch 7/50
6/6 [==============================] - 0s 29ms/step - loss: 0.8738 - mae: 0.6351 - val_loss: 0.9980 - val_mae: 0.7466
Epoch 8/50
6/6 [==============================] - 0s 34ms/step - loss: 0.8433 - mae: 0.6208

In [3]:
def calculate_fantasy_points(row, match_type):
    points = 0
    
    
    points += 4  
    points += row['Runs Scored']
    points += row['Wickets Taken'] * 25 if match_type != 'Test' else row['Wickets Taken'] * 16
    
    
    if row['Runs Scored'] == 0 and row['Balls Faced'] > 0:
        points -= 2 if match_type in ['T20', 'T10'] else 3 if match_type == 'ODI' else 4
    
    
    if row['Runs Scored'] >= 50:
        points += 8 if match_type == 'T20' else 4 if match_type in ['ODI', 'Test'] else 16
    if row['Runs Scored'] >= 100:
        points += 16 if match_type == 'T20' else 8
    
    if row['Maidens Bowled'] > 0:
        points += row['Maidens Bowled'] * (12 if match_type == 'T20' else 4 if match_type == 'ODI' else 16)
    
    if row['Wickets Taken'] >= 4:
        points += 8 if match_type == 'T20' else 4
    if row['Wickets Taken'] >= 5:
        points += 16 if match_type == 'T20' else 8
    
    
    if row['Overs Bowled'] >= 2:
        economy_rate = row['Runs Conceded'] / row['Overs Bowled']
        if economy_rate < 6:
            points += 4
        elif economy_rate > 9:
            points -= 2
    
    return points


X_all_scaled = scaler_X.transform(X)
predicted_stats_scaled = model.predict(X_all_scaled)
predicted_stats = scaler_y.inverse_transform(predicted_stats_scaled)

1/1 [==============================] - 2s 2s/step


In [4]:

predicted_df = pd.DataFrame(predicted_stats, columns=y.columns)
predicted_df['Player Name'] = data['Player Name']
predicted_df['Team'] = data['Team']
predicted_df['Player Role'] = data['Player Role']

for col in y.columns:
    if col in ['Runs Scored', 'Balls Faced', 'Balls Bowled', 'Runs Conceded']:
        predicted_df[col] = predicted_df[col].round().astype(int)
    elif col in ['Wickets Taken', 'Maidens Bowled']:
        predicted_df[col] = predicted_df[col].round(1)
    elif col == 'Overs Bowled':
        predicted_df[col] = predicted_df[col].round(1)

In [5]:

match_type = 'ODI'  
predicted_df['Predicted Fantasy Points'] = predicted_df.apply(lambda row: calculate_fantasy_points(row, match_type), axis=1)

def select_top_11_with_roles(df):
    sorted_players = df.sort_values(by='Predicted Fantasy Points', ascending=False)
    
    team = []
    roles_covered = set()
    
    for _, player in sorted_players.iterrows():
        if len(team) >= 11 and len(roles_covered) == len(label_enc.classes_):
            break
        if player['Player Role'] not in roles_covered or len(team) < 11:
            team.append(player)
            roles_covered.add(player['Player Role'])
    
    while len(team) < 11:
        for _, player in sorted_players.iterrows():
            if player not in team:
                team.append(player)
                break
    
    return pd.DataFrame(team)

In [9]:
top_players = select_top_11_with_roles(predicted_df)

print("Top 11 Players based on Predicted Fantasy Points:")
print(top_players[['Player Name', 'Team', 'Predicted Fantasy Points', 'Player Role']])

print("\nPredicted Stats for Top Players (per match):")
print(top_players[['Player Name', 'Runs Scored', 'Wickets Taken', 'Balls Faced', 'Balls Bowled', 
                   'Overs Bowled', 'Maidens Bowled', 'Runs Conceded']])


role_distribution = top_players['Player Role'].value_counts()
print(role_distribution)


Top 11 Players based on Predicted Fantasy Points:
           Player Name  Team  Predicted Fantasy Points  Player Role
20        Shreyas Iyer     0                189.000000            2
16        Sanju Samson     0                186.500000            2
12     Pathum Nissanka     1                 71.500000            0
6    Washington Sundar     0                 60.800001            3
5         Shubman Gill     0                 60.000000            0
18     Janith Liyanage     1                 56.500000            2
0          Virat Kohli     0                 50.400000            0
1         Rohit Sharma     0                 50.000000            0
27       Kuldeep Yadav     0                 47.400000            2
2         Rishabh Pant     0                 43.400000            2
17    Charith Asalanka     1                 42.000000            2
14  Maheesh Theekshana     1                 24.900000            1

Predicted Stats for Top Players (per match):
           Player Na

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, GRU, Input, Concatenate, Reshape

# Load dataset
data = pd.read_csv('data.csv')

# Encode categorical data
label_enc = LabelEncoder()
data['Player Role'] = label_enc.fit_transform(data['Player Role'])
data['Team'] = label_enc.fit_transform(data['Team'])

# Define features and target variables
features = ['Batting Average', 'Bowling Average', 'Strike Rate', 'Economy Rate', 
            'Centuries Scored', 'Half Centuries Scored', 'Ducks Scored', 
            'Wickets Taken Last Match', 'Runs Scored Last Match', 
            'Player Age', 'Player Experience', 'Player Role', 'Team']

X = data[features]
y = data[['Runs Scored', 'Wickets Taken', 'Balls Faced', 'Balls Bowled', 
          'Overs Bowled', 'Maidens Bowled', 'Runs Conceded']]

# Normalize target variables
for col in y.columns:
    y[col] = y[col] / data['Player Experience']

# Standardize features and target variables
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Create the model
def create_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    
    reshaped_input = Reshape((input_shape, 1))(input_layer)
    
    lstm = LSTM(64, return_sequences=True)(reshaped_input)
    lstm = LSTM(32)(lstm)
    
    gru = GRU(64, return_sequences=True)(reshaped_input)
    gru = GRU(32)(gru)
    
    concat = Concatenate()([lstm, gru])
    
    dense1 = Dense(64, activation='relu')(concat)
    dense2 = Dense(32, activation='relu')(dense1)
    
    output = Dense(7)(dense2)  
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=50, batch_size=4, validation_data=(X_test, y_test))

# Fantasy points calculation function
def calculate_fantasy_points(row, match_type):
    points = 0
    points += 4  # Appearance points
    points += row['Runs Scored']
    points += row['Wickets Taken'] * 25 if match_type != 'Test' else row['Wickets Taken'] * 16
    
    # Duck penalty
    if row['Runs Scored'] == 0 and row['Balls Faced'] > 0:
        points -= 2 if match_type in ['T20', 'T10'] else 3 if match_type == 'ODI' else 4
    
    # Batting bonuses
    if row['Runs Scored'] >= 50:
        points += 8 if match_type == 'T20' else 4 if match_type in ['ODI', 'Test'] else 16
    if row['Runs Scored'] >= 100:
        points += 16 if match_type == 'T20' else 8
    
    # Bowling bonuses
    if row['Maidens Bowled'] > 0:
        points += row['Maidens Bowled'] * (12 if match_type == 'T20' else 4 if match_type == 'ODI' else 16)
    if row['Wickets Taken'] >= 4:
        points += 8 if match_type == 'T20' else 4
    if row['Wickets Taken'] >= 5:
        points += 16 if match_type == 'T20' else 8
    
    # Economy rate bonuses/penalties
    if row['Overs Bowled'] >= 2:
        economy_rate = row['Runs Conceded'] / row['Overs Bowled']
        if economy_rate < 6:
            points += 4
        elif economy_rate > 9:
            points -= 2
    
    return points

# Function to select top 11 players from specific teams
def get_top_11_players_for_teams(data, team_1, team_2, match_type='ODI'):
    # Filter data for the specified teams
    team_data = data[data['Team'].isin([team_1, team_2])]
    
    # Prepare the data for prediction
    X_team = team_data[features]
    X_team_scaled = scaler_X.transform(X_team)
    
    # Predict stats for the filtered dataset
    predicted_stats_scaled = model.predict(X_team_scaled)
    predicted_stats = scaler_y.inverse_transform(predicted_stats_scaled)
    
    predicted_df = pd.DataFrame(predicted_stats, columns=y.columns)
    predicted_df['Player Name'] = team_data['Player Name'].values
    predicted_df['Team'] = team_data['Team'].values
    predicted_df['Player Role'] = team_data['Player Role'].values
    
    for col in y.columns:
        if col in ['Runs Scored', 'Balls Faced', 'Balls Bowled', 'Runs Conceded']:
            predicted_df[col] = predicted_df[col].round().astype(int)
        elif col in ['Wickets Taken', 'Maidens Bowled']:
            predicted_df[col] = predicted_df[col].round(1)
        elif col == 'Overs Bowled':
            predicted_df[col] = predicted_df[col].round(1)
    
    predicted_df['Predicted Fantasy Points'] = predicted_df.apply(lambda row: calculate_fantasy_points(row, match_type), axis=1)
    
    return select_top_11_with_roles(predicted_df)

# Top 11 player selection function with role consideration
def select_top_11_with_roles(df):
    sorted_players = df.sort_values(by='Predicted Fantasy Points', ascending=False)
    
    team = []
    roles_covered = set()
    
    for _, player in sorted_players.iterrows():
        if len(team) >= 11 and len(roles_covered) == len(label_enc.classes_):
            break
        if player['Player Role'] not in roles_covered or len(team) < 11:
            team.append(player)
            roles_covered.add(player['Player Role'])
    
    while len(team) < 11:
        for _, player in sorted_players.iterrows():
            if player not in team:
                team.append(player)
                break
    
    return pd.DataFrame(team)

# Example usage
team_1 = 'Sri Lanka'  # You need to pass the correct label encoded value or modify the code to use team names directly
team_2 = 'Australia'  # You need to pass the correct label encoded value or modify the code to use team names directly

# Filter data based on team names
team_1_encoded = label_enc.transform([team_1])[0]
team_2_encoded = label_enc.transform([team_2])[0]

top_players = get_top_11_players_for_teams(data, team_1_encoded, team_2_encoded)

print("Top 11 Players based on Predicted Fantasy Points:")
print(top_players[['Player Name', 'Team', 'Predicted Fantasy Points', 'Player Role']])

print("\nPredicted Stats for Top Players (per match):")
print(top_players[['Player Name', 'Runs Scored', 'Wickets Taken', 'Balls Faced', 'Balls Bowled', 
                   'Overs Bowled', 'Maidens Bowled', 'Runs Conceded']])

role_distribution = top_players['Player Role'].value_counts()
print(role_distribution)


C:\Users\shoya\AppData\Local\Temp\ipykernel_14564\2998751330.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[col] = y[col] / data['Player Experience']


Epoch 1/50
10/10 [==============================] - 11s 233ms/step - loss: 1.0348 - mae: 0.7467 - val_loss: 0.8628 - val_mae: 0.7715
Epoch 2/50
10/10 [==============================] - 0s 21ms/step - loss: 1.0215 - mae: 0.7344 - val_loss: 0.8793 - val_mae: 0.7771
Epoch 3/50
10/10 [==============================] - 0s 20ms/step - loss: 1.0011 - mae: 0.7177 - val_loss: 0.8972 - val_mae: 0.7810
Epoch 4/50
10/10 [==============================] - 0s 21ms/step - loss: 0.9818 - mae: 0.6981 - val_loss: 0.9074 - val_mae: 0.7789
Epoch 5/50
10/10 [==============================] - 0s 21ms/step - loss: 0.9574 - mae: 0.6730 - val_loss: 0.8632 - val_mae: 0.7627
Epoch 6/50
10/10 [==============================] - 0s 22ms/step - loss: 0.9342 - mae: 0.6576 - val_loss: 0.8208 - val_mae: 0.7356
Epoch 7/50
10/10 [==============================] - 0s 21ms/step - loss: 0.9003 - mae: 0.6317 - val_loss: 0.8332 - val_mae: 0.7410
Epoch 8/50
10/10 [==============================] - 0s 20ms/step - loss: 0.8799 -